In [7]:

# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display
import anthropic

In [8]:
load_dotenv(override=True)

anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')





In [9]:
# Using open router with any model
from openai import OpenAI

llm_client = OpenAI(
    api_key=openrouter_api_key,
    base_url="https://openrouter.ai/api/v1",
)



In [11]:
OPENAI_MODEL = "gpt-5"
CLAUDE_MODEL = "anthropic/claude-haiku-4-5"
# GROK_MODEL = "grok-4"
GEMINI_MODEL = "google/gemini-2.5-flash-lite"

In [ ]:
response = llm_client.chat.completions.create(
    model= GEMINI_MODEL,
    messages=[{"role": "user", "content": "Hi?"}],
)

print(response.choices[0].message.content)


Hi there! How can I help you today? 😊


In [ ]:
from system_info import retrieve_system_info

# Call retrieve_system_info() to gather detailed information about the current system environment,
# including OS, CPU details, toolchains, compilers, and more, and store the result in system_info.
# This will help the LLM to choose the best compiler and toolchain for the task
system_info = retrieve_system_info()
system_info

{'os': {'system': 'Linux',
  'arch': 'x86_64',
  'release': '5.10.16.3-microsoft-standard-WSL2',
  'version': '#1 SMP Fri Apr 2 22:23:49 UTC 2021',
  'kernel': '5.10.16.3-microsoft-standard-WSL2',
  'distro': {'name': 'Ubuntu 24.04.1 LTS', 'version': '24.04'},
  'wsl': True,
  'rosetta2_translated': False,
  'target_triple': 'x86_64-linux-gnu'},
 'package_managers': ['apt'],
 'cpu': {'brand': 'Intel(R) Core(TM) i7-10610U CPU @ 1.80GHz',
  'cores_logical': 8,
  'cores_physical': 4,
  'simd': ['AVX', 'AVX2', 'FMA', 'SSE4_2']},
 'toolchain': {'compilers': {'gcc': 'gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0',
   'g++': 'g++ (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0',
   'clang': '',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 4.3'},
  'linkers': {'ld_lld': ''}}}

In [12]:
# A system prompt for the code generator 

message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

response = llm_client.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

You’re already set up. Your system has g++ 13.3.0 installed on Ubuntu 24.04 (WSL2), so you don’t need to install anything to compile and run a single C++ file.

Recommended commands for fastest runtime on your machine:
- This uses native CPU optimizations, link-time optimization, and disables asserts.

Use these in your Python code:
- compile_command:
  ["g++", "-std=c++20", "-Ofast", "-march=native", "-flto", "-DNDEBUG", "main.cpp", "-o", "main"]
- run_command:
  ["./main"]

Notes:
- -Ofast may change floating-point behavior for speed. If you need strict IEEE/standard-compliant math, replace -Ofast with -O3.
- -march=native produces a binary optimized for this CPU; it may not run on different machines. If you need portability, omit -march=native.

Optional (only if you ever need to install g++):
- sudo apt update
- sudo apt install -y build-essential

In [30]:
compile_command = ["g++", "-std=c++20", "-Ofast", "-march=native", "-flto", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]

In [16]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [17]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]
 

In [18]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [19]:
def port(client, model, python):
    # function that will port the python code to C++
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [20]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [22]:
def run_python(code):
    #function that will run the python code
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [23]:
run_python(pi)

Result: 3.141592656089
Execution Time: 23.006779 seconds


In [26]:
port(llm_client, OPENAI_MODEL, pi)

In [28]:
# Use the commands from GPT 5

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [31]:
compile_and_run()

Result: 3.141592662604
Execution Time: 0.394998 seconds

Result: 3.141592662604
Execution Time: 0.393660 seconds

Result: 3.141592662604
Execution Time: 0.386548 seconds



In [ ]:
# Trying with GPT 4o
port(anthropic, CLAUDE_MODEL, pi)
compile_and_run()